# Code to map postcodes to shp files

## May - June 2020

#### Postcode definition file (version 2020May) downloaded from UK Data Service https://borders.ukdataservice.ac.uk/pcluts.html

##### Downloaded: 13 June 2020 (Previous 19 May 2020)
##### Version: 2020may (Previous 2020feb)

##### Column names are due to change in May 2020 version.

## Import libraries

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import epydemiology as epy
import geopandas as gpd
from pathlib import Path
import glob
import matplotlib.pyplot as plt
import os
from shapely.geometry import Point

## Deal with postcode data

### Load csv file into dataframe

In [ ]:
# CSV files don't contain headers. The names of the column headings are stored
# in a file called Code-Point_Open_Column_Headers.csv. Retrieve list of columns from file.
#phjPathToDoc = './data/postcode/onspd/pcluts_2020feb'
#phjFilename = '/ONSPD_FEB_2020_UK.csv'

phjPathToDoc = './data/postcode/onspd/pcluts_2020may'
phjFilename = '/ONSPD_MAY_2020_UK.csv'

a

phjPostcodeDF = pd.read_csv(Path('/'.join([phjPathToDoc,phjFilename])))

print(phjPostcodeDF.dtypes)
print('\n')
print(phjPostcodeDF)

In [ ]:
phjPostcodeDF = phjPostcodeDF[['pcd','pcd2','pcds','ctry','oseast1m','osnrth1m','lat','long']].copy()

print(phjPostcodeDF.dtypes)
print('\n')
print(phjPostcodeDF)

### Convert to geodataframe

#### Example (choose one or other method below, either oseast1m/osnrth1m or long/lat)

File contains both longitude ('long') and latitude ('lat'), and easting ('oseast1m') and northing ('osnrth1m') columns. The longitude and latitude points use WGS84 (espg 4326) whilst the easting and northing points use OSGB96 projection (espg 27700) for GB postcodes and (presumably) epsg 2157 for points in Northern Ireland (see below). Either of these columns could be used to create a geometry and converted between the two crs systems. As an example, in the following cell the 'long' and 'lat' columns are converted to a point geometry (with epsg = 4326) in a geopandas geodataframe and the crs converted to epsg 27700.


CRS used in Ireland

EPSG:29901 OSNI 1952 / Irish National Grid. Not used in Republic of Ireland. Replaced in 1975 by TM75 / Irish Grid (CRS code 29903) (see https://epsg.io/29901)

EPSG:29902 TM65 / Irish Grid -- Ireland. Not used in Northern Ireland. Replaced by TM75 / Irish Grid (code 29903) in 1975 (see https://epsg.io/29902)

EPSG:29903 TM75 / Irish Grid. Replaces both OSNI 1952 / Irish National Grid (code 29901) and TM65 / Irish Grid (code 29902) from 1975. Replaced by IRENET95 / Irish Transverse Mercator (code 2157) from 1/1/2001. (see https://epsg.io/29903)

EPSG:2157 IRENET95 / Irish Transverse Mercator Replaces TM75 / Irish Grid (code 29903) from 1/1/2001. (see https://epsg.io/2157)

##### Create geodataframe containing easting and northing data only

In [ ]:
phjPostcodeGDF = gpd.GeoDataFrame(phjPostcodeDF.drop(['oseast1m', 'osnrth1m'], axis = 1),
                                  crs = {'init': 'epsg:27700'},
                                  geometry = [Point(xy) for xy in zip(phjPostcodeDF['oseast1m'],
                                                                      phjPostcodeDF['osnrth1m'])])

# Newer version of geopandas may use
#phjPostcodeGDF = gpd.GeoDataFrame(phjPostcodeDF,
#                                  crs={'init': 'epsg:4326'},
#                                  geometry=gpd.points_from_xy(phjPostcodeDF['EA'],
#                                                              phjPostcodeDF['NO']))

print('CRS')
print('---')
print(phjPostcodeGDF.crs)
print('\n')
print('GeoPandas DataFrame with postcode point geometry')
print('------------------------------------------------')
print(phjPostcodeGDF)

In [ ]:
print(phjPostcodeGDF.dtypes)
print('\n')
print(phjPostcodeGDF['ctry'].value_counts())

In [ ]:
# Plot postcode points
# --------------------

fig, ax = plt.subplots(1,
                       figsize = (8,10))

ax.set_axis_off()

phjPostcodeGDF.plot(ax = ax,
                    color = 'red',
                    markersize = 10)

fig.suptitle('Postcode plot based on converting easting-northing data to epsg 27700')

plt.show()

##### Create geodataframe containing longitude and latitude data only

N.B. this cell replaces phjPostcodeGDF created in previous cell.

In [ ]:
# Create a point geometry and drop the original 'long' and 'lat' columns
#phjPostcodeGDF = gpd.GeoDataFrame(phjPostcodeDF.drop(['long', 'lat'], axis = 1),
#                                  crs = {'init': 'epsg:4326'},
#                                  geometry = [Point(xy) for xy in zip(phjPostcodeDF['long'],
#                                                                      phjPostcodeDF['lat'])])

# Create a point geometry and retain the original 'long' and 'lat' columns
phjPostcodeGDF = gpd.GeoDataFrame(phjPostcodeDF,
                                  crs = {'init': 'epsg:4326'},
                                  geometry = [Point(xy) for xy in zip(phjPostcodeDF['long'],
                                                                      phjPostcodeDF['lat'])])



# Newer version of geopandas may use
#phjPostcodeGDF = gpd.GeoDataFrame(phjPostcodeDF,
#                                  crs={'init': 'epsg:4326'},
#                                  geometry=gpd.points_from_xy(phjPostcodeDF['EA'],
#                                                              phjPostcodeDF['NO']))

phjPostcodeGDF = phjPostcodeGDF.to_crs(epsg=27700)

print('CRS')
print('---')
print(phjPostcodeGDF.crs)
print('\n')
print('GeoPandas DataFrame with postcode point geometry')
print('------------------------------------------------')
print(phjPostcodeGDF)

In [ ]:
# Plot postcode points
# --------------------

fig, ax = plt.subplots(1,
                       figsize = (8,10))

ax.set_axis_off()

phjPostcodeGDF.plot(ax = ax,
                    color = 'red',
                    markersize = 10)

fig.suptitle('Postcode plot based on converting long-lat data to epsg 27700')

plt.show()

IMPORTANT NOTICE

There are 2 geographical variables included in the ONS postcode file, namely 'long' and 'lat', and 'oseast1m' and 'osnrth1m'. It is relatively easy to convert the long/lat data to epsg 27700 projection for plotting on a map as shown in the second map above. However, the easting/northing data cannot but used to plot data directly (even though, on first look, the data appears to be epsg 27700) because the data for postcodes in Northern Ireland use a different co-ordinate system (presumably epsg 2157). The results can be seen in the first map above where postcode points in Northern Ireland are plotted over North Wales and the Irish Sea.

### Remove postcodes from Northern Ireland (and IoM and Channel Islands)

Retain postcodes only from England, Wales and Scotland; exclude postcodes from Northern Ireland, Isle of Man and Channel Islands

In [ ]:
# Read country codes from Country names and codes UK as at 08_12.csv

phjCtryDefPath = './data/postcode/onspd/pcluts_2020may/Documents'
phjCtryDefFilename = 'Country names and codes UK as at 08_12.csv'

phjCtryDefDF = pd.read_csv(Path('/'.join([phjCtryDefPath,phjCtryDefFilename])))

print(phjCtryDefDF)

In [ ]:
# Create list of country codes
phjCtryCodesList = list(phjCtryDefDF.loc[phjCtryDefDF['CTRY12NM'].isin(['England','Scotland','Wales']),'CTRY12CD'])

print('List of country codes')
print('---------------------')
print(phjCtryCodesList)

In [ ]:
# Retain Eng, Scot and Wal postcodes only
phjPostcodeGDF = phjPostcodeGDF[phjPostcodeGDF['ctry'].isin(phjCtryCodesList)].copy()

In [ ]:
# Plot postcode points
# --------------------

fig, ax = plt.subplots(1,
                       figsize = (8,10))

ax.set_axis_off()

phjPostcodeGDF.plot(ax = ax,
                    color = 'red',
                    markersize = 10)

fig.suptitle('GB-only postcode plot based on converting easting-northing data to epsg 27700')

plt.show()

### Print postcodes where long/lat data is missing

In [ ]:
print('Postcodes with missing long/lat data')
print('------------------------------------')
print(phjPostcodeGDF.loc[(phjPostcodeGDF['long'].isnull()) |
                         (phjPostcodeGDF['lat'].isnull()),:])

### Remove unnecessary columns

In [ ]:
phjPostcodeGDF = phjPostcodeGDF[['pcd','geometry']].copy()

print(phjPostcodeGDF)

## Deal with shape file county boundary definitions

##### There are 2 shp files commonly used in the SEDA group that define: i) county boundaries that are used by CPH numbers (more or less); and ii) county boundaries used to display results

### i. Open .shp file containing definitions of county boundaries used by CPH numbers

Shape file opened in GeoPandas.

In [ ]:
phjPathToShp = './data/shp/CountyCPH_SPIDA'
phjShpFilename = 'CountyCPH_SPIDA.shp'

phjCPHCountyShpGDF = gpd.read_file(Path('/'.join([phjPathToShp,phjShpFilename])))

print('Shape file defining county boundaries')
print('-------------------------------------')
print(phjCPHCountyShpGDF)
print('\n')
print('CRS of shape file')
print('-----------------')
print(phjCPHCountyShpGDF.crs)

#### Plot county boundaries

In [ ]:
phjCPHCountyShpGDF.plot()

#### Example of changing the shapefile CRS

It is easy to change the CRS of the shape file if required.

In [ ]:
#phjCPHCountyShpGDF = phjCPHCountyShpGDF.to_crs(epsg=4326)

#phjCPHCountyShpGDF.plot()

#### Example of plotting individual points on map of county boundaries

In [ ]:
phjExamplePostcodeList = ['CH5 4HE','NP4 5DG','CH647TE','W1A 1AA','KT153NB']

In [ ]:
fig, ax = plt.subplots(1,
                       figsize = (16,20))

ax = phjCPHCountyShpGDF.plot(ax = ax,
                             color = 'white',
                             edgecolor = 'black')

ax.set_axis_off()

phjPostcodeGDF[phjPostcodeGDF['pcd'].isin(phjExamplePostcodeList)].plot(ax = ax,
                                                                       color = 'red')

plt.show()

#### Spatial join postcode geodataframe with county shapefile geodataframe to produce dataframe that links postcode with CPHCounty names

In [ ]:
phjPostcodeCPHCountyGDF = gpd.sjoin(phjPostcodeGDF[['pcd','geometry']],
                                    phjCPHCountyShpGDF[['CTYID','CTY_Name','geometry']],
                                    op = 'within',
                                    how = 'left')

# Remove the column entitled 'index_right'
phjPostcodeCPHCountyGDF = phjPostcodeCPHCountyGDF[[c for c in phjPostcodeCPHCountyGDF if c not in ['index_right']]].rename(columns = {'CTYID':'CPH_CTYID',
                                                                                                                                      'CTY_Name':'CPH_CTY_Name'}).copy()

print('Postcode-CPHCounty lookup table')
print('-------------------------------')
print(phjPostcodeCPHCountyGDF)

#### Check example postcodes are in the correct county

In [ ]:
print(phjPostcodeCPHCountyGDF[phjPostcodeCPHCountyGDF['pcd'].isin(phjExamplePostcodeList)])

### ii. Open .shp file containing county definitions used to report SIU outputs

In [ ]:
pwd

In [ ]:
phjPathToShp = './data/shp/County2000'
phjShpFilename = 'County2000.shp'

phjCounty2000ShpGDF = gpd.read_file(Path('/'.join([phjPathToShp,phjShpFilename])))

print('Shape file defining county boundaries')
print('-------------------------------------')
print(phjCounty2000ShpGDF)
print('\n')
print('CRS of shape file')
print('-----------------')
print(phjCounty2000ShpGDF.crs)

#### Spatial join postcode geodataframe with county shapefile geodataframe to produce dataframe that links postcode with County2000 names

In [ ]:
phjPostcodeCounty2000GDF = gpd.sjoin(phjPostcodeGDF[['pcd','geometry']],
                                     phjCounty2000ShpGDF[['OBJECTID_1','IDENTITY','CORRECT_CO','geometry']],
                                     op = 'within',
                                     how = 'left')

# Remove the column entitled 'index_right'
phjPostcodeCounty2000GDF = phjPostcodeCounty2000GDF[[c for c in phjPostcodeCounty2000GDF if c not in ['index_right']]].rename(columns = {'OBJECTID_1':'CTY2000_OBJECTID',
                                                                                                                                         'IDENTITY':'CTY2000_IDENTITY',
                                                                                                                                         'CORRECT_CO':'CTY2000_CORRECT_CO'}).copy()

print('Postcode-County2000 lookup table')
print('-------------------------------')
print(phjPostcodeCounty2000GDF)

## Combine postcode-county dataframes

#### In original version, 'geometry' column removed but subsequently decided that needed to be able to plot unmapped values therefore decided to retain 'geometry' column

In [ ]:
#phjPostcodeCountyDF = pd.merge(phjPostcodeCPHCountyGDF[[c for c in phjPostcodeCPHCountyGDF if c not in ['geometry']]],
#                               phjPostcodeCounty2000GDF[[c for c in phjPostcodeCounty2000GDF if c not in ['geometry']]],
#                               on = 'pcd')

phjPostcodeCountyGDF = pd.merge(phjPostcodeCPHCountyGDF,
                                phjPostcodeCounty2000GDF[[c for c in phjPostcodeCounty2000GDF if c not in ['geometry']]],
                                on = 'pcd')


print(phjPostcodeCountyGDF)

#### Check example postcodes have the correct county definitions

In [ ]:
print(phjPostcodeCountyGDF[phjPostcodeCountyGDF['pcd'].isin(phjExamplePostcodeList)])

In [ ]:
with pd.option_context('display.max_rows', 100, 'display.max_columns', 10):
    print(phjPostcodeCountyGDF['CTY2000_CORRECT_CO'].value_counts(dropna = False))

## Postcodes with missing county definitions

### County2000 definitions only

In [ ]:
print(phjPostcodeCountyGDF.loc[phjPostcodeCountyGDF['CTY2000_CORRECT_CO'].isnull(),:])

### Both CPH County and County2000 definitions

In [ ]:
print(phjPostcodeCountyGDF.loc[(phjPostcodeCountyGDF['CPH_CTY_Name'].isnull()) |
                               (phjPostcodeCountyGDF['CTY2000_CORRECT_CO'].isnull()),:])

### Add column to indicate with either or both county definitions are null

In [ ]:
phjPostcodeCountyGDF['nullCounty'] = 'neither_null'

phjPostcodeCountyGDF.loc[(phjPostcodeCountyGDF['CPH_CTY_Name'].isnull()) &
                         (phjPostcodeCountyGDF['CTY2000_CORRECT_CO'].notnull()),'nullCounty'] = 'cph_cty_name_only_null'

phjPostcodeCountyGDF.loc[(phjPostcodeCountyGDF['CPH_CTY_Name'].notnull()) &
                         (phjPostcodeCountyGDF['CTY2000_CORRECT_CO'].isnull()),'nullCounty'] = 'cty2000_correct_co_only_null'

phjPostcodeCountyGDF.loc[(phjPostcodeCountyGDF['CPH_CTY_Name'].isnull()) &
                         (phjPostcodeCountyGDF['CTY2000_CORRECT_CO'].isnull()),'nullCounty'] = 'both_null'

print('Number of postcodes not mapped to county defintions')
print('---------------------------------------------------')
print(phjPostcodeCountyGDF['nullCounty'].value_counts())

### Print postcodes with missing county

In [ ]:
print('Postcodes with missing CPH county definitions only')
print('--------------------------------------------------')
print(phjPostcodeCountyGDF.loc[phjPostcodeCountyGDF['nullCounty'].eq('cph_cty_name_only_null'),:])
print('\n')

print('Postcodes with missing County2000 definition only')
print('-------------------------------------------------')
print(phjPostcodeCountyGDF.loc[phjPostcodeCountyGDF['nullCounty'].eq('cty2000_correct_co_only_null'),:])

### Plot missing postcodes

In [ ]:
fig, ax = plt.subplots(1,
                       figsize = (16,20))

ax = phjCPHCountyShpGDF.plot(ax = ax,
                             color = 'white',
                             edgecolor = 'gray')

ax.set_axis_off()

phjPostcodeCountyGDF.loc[phjPostcodeCountyGDF['nullCounty'].eq('cty2000_correct_co_only_null'),['pcd','geometry']].plot(ax = ax,
                                                                                                                   color = 'red',
                                                                                                                       markersize = 20)

phjPostcodeCountyGDF.loc[phjPostcodeCountyGDF['nullCounty'].eq('cph_cty_name_only_null'),['pcd','geometry']].plot(ax = ax,
                                                                                                             color = 'blue',
                                                                                                             markersize = 20)

phjPostcodeCountyGDF.loc[phjPostcodeCountyGDF['nullCounty'].eq('both_null'),['pcd','geometry']].plot(ax = ax,
                                                                                                color = 'purple',
                                                                                                markersize = 20)

plt.show()

## Write to CSV file

In [ ]:
phjPathToCSV = '.'
phjCSVFilename = 'ONSPDPostcodeToShpCounties.csv'

phjPostcodeCountyGDF[[c for c in phjPostcodeCountyGDF if c not in ['geometry']]].to_csv(Path('/'.join([phjPathToCSV,phjCSVFilename])),
                           index = False)

print('Done')